# 0 Loading packages <a name="loading"></a> 

In [1]:
import sys
print(sys.executable)
print(sys.path)
print(sys.version)

/home2/s418610/.conda/envs/py37_res_PAGA/bin/python
['/home2/s418610/jupyter_notebooks/03.Invivo_neuronal_reprogramming/Manuscript/to_GitHub', '', '/cm/shared/apps/cnvkit/0.9.5/lib/python3.6/site-packages', '/cm/shared/apps/python/3.6.4-anaconda/lib', '/cm/shared/apps/jupyterkernel-matlab/0.16.1/lib/python3.6/site-packages', '/cm/shared/apps/python/2.7.x-anaconda/lib', '/home2/s418610/.conda/envs/py37_res_PAGA/lib/python37.zip', '/home2/s418610/.conda/envs/py37_res_PAGA/lib/python3.7', '/home2/s418610/.conda/envs/py37_res_PAGA/lib/python3.7/lib-dynload', '/home2/s418610/.local/lib/python3.7/site-packages', '/home2/s418610/.conda/envs/py37_res_PAGA/lib/python3.7/site-packages', '/home2/s418610/.conda/envs/py37_res_PAGA/lib/python3.7/site-packages/fa2-0.3.5-py3.7-linux-x86_64.egg', '/home2/s418610/.conda/envs/py37_res_PAGA/lib/python3.7/site-packages/IPython/extensions', '/home2/s418610/.ipython']
3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [2]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
import scipy.sparse
import sklearn.manifold
import statsmodels.api as sm

# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from pyclustering.cluster.kmedoids import kmedoids

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# import seaborn as sb

import scanpy as sc
import harmonypy as hm

In [3]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3

print('numpy', np.__version__)
print('pandas', pd.__version__)
print('scipy', scipy.__version__)
print('sklearn', sklearn.__version__)
print('statsmodels', sm.__version__)
print('matplotlib', mpl.__version__)
print('seaborn', sns.__version__)

sc.logging.print_versions()

!date +%F

numpy 1.17.2
pandas 0.25.2
scipy 1.3.1
sklearn 0.21.3
statsmodels 0.10.1
matplotlib 3.3.2
seaborn 0.9.0
scanpy==1.4.5.post1 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.2 scipy==1.3.1 pandas==0.25.2 scikit-learn==0.21.3 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1
2021-03-05


In [4]:
import anndata
import pickle

In [5]:
import statsmodels.api as sm
lowess = sm.nonparametric.lowess

In [6]:
from scipy.stats import chi2_contingency

In [7]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [8]:
import os

# set a working directory for saving plots
wdir = "/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/99.Manuscript/Figures_v2"
# wdir = "/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/99.Manuscript/Figures/Figure2/"
os.chdir( wdir )

In [9]:
# A sub-directory for saving intermediate data for organized pipeline v1
data_dir = '/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/Cluster_with_10X_20k/data/10x_remapped/PipelineOrganized_v1/'

In [10]:
neural_data_dir = data_dir + 'NeuralOnly/'

In [11]:
import statsmodels.api as sm
lowess = sm.nonparametric.lowess

In [12]:
from scipy.stats import chi2_contingency

In [13]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

# 1 Load data

### Gene cluster data

In [14]:
gene_data_dir = '/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/Cluster_with_10X_20k/data/10x_remapped/PipelineOrganized_v1/Gene_clusters/'

In [15]:
input_path = gene_data_dir + 'adata_genes_Manuscript_v2'
with open(input_path, 'rb') as input_file:
 
    adata_genes = pickle.load(input_file)

In [16]:
adata_genes

AnnData object with n_obs × n_vars = 2469 × 4601 
    obs: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'p_val_adj', 'n_counts', 'n_cells_experimental', 'louvain', 'log10_n_cells_experimental'
    var: 'library_name', 'batch', 'lenti_count', 'lenti_log1p_count', 'tdT_count', 'tdT_log1p_count', 'n_counts', 'log_counts', 'n_genes', 'mt_count', 'mt_frac', 'n_counts_all', 'louvain', 'dpt_pseudotime', 'rank_pseudotime', 'Rpl32', 'Egfr'
    uns: 'pca', 'neighbors', 'louvain', 'paga', 'louvain_sizes', 'louvain_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [17]:
# How many genes are in each cluster? 
louvain_clusters_counter = adata_genes.obs['louvain'].value_counts()
louvain_clusters_counter

0     582
1     313
2     309
3     302
4     240
5     218
6     135
7     131
8      91
9      76
10     56
11     16
Name: louvain, dtype: int64

In [18]:
adata_genes.obs

,gene_ids,feature_types,genome,n_cells,p_val_adj,n_counts,n_cells_experimental,louvain,log10_n_cells_experimental
Rgs20,ENSMUSG00000002459,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,1062,NaN,951.583130,1062,0,3.026533
St18,ENSMUSG00000033740,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,276,NaN,239.741074,276,2,2.442480
Rrs1,ENSMUSG00000061024,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,966,NaN,806.224243,966,3,2.985426
Adhfe1,ENSMUSG00000025911,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,547,NaN,427.636810,547,0,2.738781
Mybl1,ENSMUSG00000025912,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,140,NaN,109.949699,140,1,2.149219
...,...,...,...,...,...,...,...,...,...
Eif3a,ENSMUSG00000024991,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,2458,NaN,2743.859375,2257,1,3.353724
mt-Nd4l,ENSMUSG00000065947,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,2930,NaN,3241.866699,2022,0,3.305996
mt-Nd5,ENSMUSG00000064367,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,3469,NaN,5341.198242,1945,0,3.289143
Lenti-hGfa-Dlx2-IRES-GFP,Lenti-hGfa-Dlx2-IRES-GFP,Gene Expression,mm10-3.0.0-Lenti-Dlx2_tdTomato,2411,NaN,3311.665039,2140,7,3.330617


# 2 Compare genes from Lindtner et al

In [85]:
df_overlap = pd.DataFrame(index=adata_genes.obs['louvain'].cat.categories, columns=['num_genes_in_cluster', 'down_genes', 'up_genes', 'aRE_genes', 'rRE_genes'])

In [86]:
df_overlap

,num_genes_in_cluster,down_genes,up_genes,aRE_genes,rRE_genes
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


## 2.1 Compare with Lindtner et al. DEGs in DLX KO

In [38]:
Lindtner_genes = pd.read_csv('/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/10.Comparison_w_Lindtner2019/DE_genes/DE_genes_Lindtner2019.csv')

In [39]:
down_genes = Lindtner_genes['Gene'][(Lindtner_genes['logFC'] < 0).tolist()].tolist()

In [40]:
up_genes = Lindtner_genes['Gene'][(Lindtner_genes['logFC'] > 0).tolist()].tolist()

In [41]:
len(down_genes)

181

In [42]:
len(up_genes)

147

### 2.1.1 Down genes

In [87]:
for current_cluster in adata_genes.obs['louvain'].cat.categories: 

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    df_overlap.loc[current_cluster, 'down_genes'] = len(intersection(gene_cluster, down_genes))
    df_overlap.loc[current_cluster, 'num_genes_in_cluster'] = len(gene_cluster)

### 2.1.2 Up genes

In [88]:
for current_cluster in adata_genes.obs['louvain'].cat.categories: 

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    df_overlap.loc[current_cluster, 'up_genes'] = len(intersection(gene_cluster, up_genes))

## 2.2 Compare with Lindtner et al. aRE- and rRE-associated genes

In [89]:
Lindtner_peaks = pd.read_csv('/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/10.Comparison_w_Lindtner2019/REs/Dlx_ChIP-seq_peaks.csv')

/home2/s418610/.conda/envs/py37_res_PAGA/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18,20,121,123,124,125,126,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [90]:
Lindtner_peaks

,Gene,PeakID,seqnames,start,end,width,nearest.TSS,dist.nearest.TSS,Feature,VertebratePhastcons,...,LGE.MZ,MGE.VZ,MGE.SVZ,MGE.MZ,POA.VZ,POA.SVZ,POA.MZ,Se.VZ,Se.SVZ,Se.MZ
0,mKIAA1889,Merged.1,chr1,3084850,3085335,486,mKIAA1889_UCSC,120376,Intergenic,262.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mKIAA1889,Merged.2,chr1,3200383,3200701,319,mKIAA1889_UCSC,5010,GeneBody,470.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mKIAA1889,Merged.3,chr1,3279440,3279901,462,mKIAA1889_UCSC,73726,GeneBody,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Rp1,Merged.4,chr1,4322611,4322865,255,Rp1_RefSeq,27528,GeneBody,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mrpl15,Merged.5,chr1,4775680,4776335,656,Mrpl15_RefSeq,0,TSS,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16914,Tlr7,Merged.17127,chrX,163759141,163759683,543,Tlr7_UCSC,8346,GeneBody,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16915,Frmpd4,Merged.17128,chrX,164211555,164211920,366,Frmpd4_RefSeq,81087,GeneBody,285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16916,Mid1,Merged.17129,chrX,166275796,166276628,833,Mid1_RefSeq,9704,GeneBody,303.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16917,Fxy,Merged.17130,chrX,166407669,166414496,6828,Fxy_UCSC,0,TSS,292.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
aRE_regions = Lindtner_peaks[Lindtner_peaks['a.RE.bdg'] == 1]

aRE_genes = aRE_regions['Gene']

aRE_genes = aRE_genes.unique().tolist()

rRE_regions = Lindtner_peaks[Lindtner_peaks['r.RE.bdg'] == 1]

rRE_genes = rRE_regions['Gene']

rRE_genes = rRE_genes.unique().tolist()

In [92]:
len(aRE_genes)

447

In [93]:
len(rRE_genes)

418

### 2.2.1 aRE-associated genes

In [94]:
for current_cluster in adata_genes.obs['louvain'].cat.categories: 

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    df_overlap.loc[current_cluster, 'aRE_genes'] = len(intersection(gene_cluster, aRE_genes))

### 2.2.2 rRE-associated genes

In [95]:
for current_cluster in adata_genes.obs['louvain'].cat.categories: 

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    df_overlap.loc[current_cluster, 'rRE_genes'] = len(intersection(gene_cluster, rRE_genes))

In [96]:
df_overlap

,num_genes_in_cluster,down_genes,up_genes,aRE_genes,rRE_genes
0,582,2,14,10,18
1,313,2,7,5,13
2,309,7,3,11,13
3,302,0,1,5,11
4,240,24,1,27,20
5,218,2,1,5,7
6,135,1,0,2,0
7,131,5,2,3,4
8,91,0,0,0,1
9,76,1,1,0,1


In [98]:
df_overlap.to_csv('Overlap_summary.csv')

## 2.3 Compare with Lindtner et al. DLX-bound genes

In [53]:
Lindtner_peaks = pd.read_csv('/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/10.Comparison_w_Lindtner2019/REs/Dlx_ChIP-seq_peaks.csv')

In [54]:
Lindtner_peaks

,Gene,PeakID,seqnames,start,end,width,nearest.TSS,dist.nearest.TSS,Feature,VertebratePhastcons,...,LGE.MZ,MGE.VZ,MGE.SVZ,MGE.MZ,POA.VZ,POA.SVZ,POA.MZ,Se.VZ,Se.SVZ,Se.MZ
0,mKIAA1889,Merged.1,chr1,3084850,3085335,486,mKIAA1889_UCSC,120376,Intergenic,262.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mKIAA1889,Merged.2,chr1,3200383,3200701,319,mKIAA1889_UCSC,5010,GeneBody,470.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mKIAA1889,Merged.3,chr1,3279440,3279901,462,mKIAA1889_UCSC,73726,GeneBody,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Rp1,Merged.4,chr1,4322611,4322865,255,Rp1_RefSeq,27528,GeneBody,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mrpl15,Merged.5,chr1,4775680,4776335,656,Mrpl15_RefSeq,0,TSS,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16914,Tlr7,Merged.17127,chrX,163759141,163759683,543,Tlr7_UCSC,8346,GeneBody,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16915,Frmpd4,Merged.17128,chrX,164211555,164211920,366,Frmpd4_RefSeq,81087,GeneBody,285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16916,Mid1,Merged.17129,chrX,166275796,166276628,833,Mid1_RefSeq,9704,GeneBody,303.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16917,Fxy,Merged.17130,chrX,166407669,166414496,6828,Fxy_UCSC,0,TSS,292.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df_fraction_overlap = df_overlap.T/adata_genes.obs['louvain'].value_counts().values

In [56]:
df_fraction_overlap = df_fraction_overlap.T

In [57]:
df_fraction_overlap

,down_genes,up_genes,aRE_genes,rRE_genes
0,0.003436,0.024055,0.017182,0.030928
1,0.006390,0.022364,0.015974,0.041534
2,0.022654,0.009709,0.035599,0.042071
3,0.000000,0.003311,0.016556,0.036424
4,0.100000,0.004167,0.112500,0.083333
5,0.009174,0.004587,0.022936,0.032110
6,0.007407,0.000000,0.014815,0.000000
7,0.038168,0.015267,0.022901,0.030534
8,0.000000,0.000000,0.000000,0.010989
9,0.013158,0.013158,0.000000,0.013158


# 3 Print important overlap genes

In [83]:
# current_cluster = '4'
for current_cluster in range(12):
    current_cluster = str(current_cluster)

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    overlap_down = intersection(gene_cluster, down_genes)

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    overlap_up = intersection(gene_cluster, up_genes)

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    overlap_aRE = intersection(gene_cluster, aRE_genes)

    gene_cluster = adata_genes.obs.loc[adata_genes.obs['louvain']==current_cluster].index

    overlap_rRE = intersection(gene_cluster, rRE_genes)

    overlap_down = pd.DataFrame(data=overlap_down)

    overlap_down.to_csv('Cluster'+current_cluster+'Genes_overlap_down.csv')

    overlap_up = pd.DataFrame(data=overlap_up)

    overlap_up.to_csv('Cluster'+current_cluster+'Genes_overlap_up.csv')

    overlap_aRE = pd.DataFrame(data=overlap_aRE)

    overlap_aRE.to_csv('Cluster'+current_cluster+'Genes_overlap_aRE.csv')

    overlap_rRE = pd.DataFrame(data=overlap_rRE)

    overlap_rRE.to_csv('Cluster'+current_cluster+'Genes_overlap_rRE.csv')